In [1]:
import numpy as np
import py4dgeo
import pickle
from tqdm import tqdm
import os, sys
sys.path.append('../src')
import helper, classes

In [2]:
# _cloudcompare_param_mapping = {
#     "normalscale": "normal_radii",
#     "registrationerror": "reg_error",
#     "searchdepth": "max_distance",
#     "searchscale": "cyl_radii",
#     "usemedian": "robust_aggr",
# }

##### Create class wise point clouds 

In [8]:
def laspy_to_np_array(laspy_points):
    x = laspy_points['x']
    y = laspy_points['y']
    z = laspy_points['z']
    return np.column_stack((x, y, z))

def class_split_pc(points_all_classes, type=None):
    if type == 'real':
        labels = points_all_classes['classification']
        ordered_classes_dict = classes.CLASSES_FOR_M3C2_REAL
    elif type == 'synth':
        labels = points_all_classes.semantic_tags
        ordered_classes_dict = classes.CLASSES_FOR_M3C2_SYNTH
    else:
        return
    
    class_wise_points = []
    for class_number in ordered_classes_dict:
        class_indices = np.isin(labels, class_number)
        class_points = points_all_classes[class_indices]
        class_points_np = laspy_to_np_array(class_points)
        class_wise_points.append(class_points_np)
    
    return class_wise_points

In [4]:
synth_pc_path = '/home/Meins/Uni/TUM/SS23/Data Lab/Data Sets/Synthetic/Val_1 - Cloud.las'
real_pc_path = '/home/Meins/Uni/TUM/SS23/Data Lab/Labelling/Label-Datasets/train/Train1 - labelled.las'
real_points_all_classes, synth_points_all_classes = helper.import_and_prepare_point_clouds(real_pc_path, synth_pc_path, shift_real=True, flip_synth=True, crop=True)

In [9]:
real_points_class_wise = class_split_pc(real_points_all_classes, type='real')
synth_points_class_wise = class_split_pc(synth_points_all_classes, type='synth')

In [13]:
print(real_points_class_wise[4])

[[ 4191.07568 -1297.33325   417.89731]
 [ 4191.07568 -1297.33325   417.9053 ]
 [ 4191.07568 -1297.3335    417.91559]
 ...
 [ 4160.1416  -1251.56421   422.47769]
 [ 4160.14014 -1251.5647    422.4552 ]
 [ 4159.95605 -1251.4563    422.86859]]


In [ ]:
#m3c2 = py4dgeo.CloudCompareM3C2.calculate_distances(epoch1=, epoch2=)
m3c2 = py4dgeo.M3C2(epochs=(epoch1, epoch2),
    corepoints=corepoints,
    cyl_radii=(2.0,),
    normal_radii=(0.5, 1.0, 2.0)
)